# Set phrase identification using ngram frequency and normalized pointwise mutual information (NPMI)
The algorithm builds a distribution of ngrams with length between predefined min and max values, identifies an ngram with maximum NPMI and merges it into a single token. This operation is repeated until predefined min NPMI is reached.

To this end, I use modified definition of NMPI for several tokens:

log(p(x1...xn)/p(x1)...p(xn)) / [-log((p(x1...xn)) * (n-1)]

(n-1) is needed to make sure NMPI = 1 for tokens that are always used together.

However, I think this definition needs a bias to encourage joining longer ngrams, otherwises it mostly merges bigrams. So I introduced a parameter alpha to control bias towards longer or shorter ngrams and replaced (n-1) with (n-1)**alpha. The parameter works in reverse: alpha = 0 biases the algorithm towards longer ngrams; alpha = 1, towards shorter ngrams.

Another tunable hyperparameter is the number of most common ngrams considered at every iteration (N).

This version tries to take both NPMI and relative ngram frequency into consideration. Relative frequency is defined as the frequency of a particular engram divided by the maximum ngram frequency in this dataset. The combined parameter is:

colloc = w * rel_freq + npmi

where w is weight controling contribution of frequency.

### Batching
For long files, I use batching: the next ngram to be merged is found in a batch rather than in the whole file and then it is merged in the whole file.

In [1]:
import nltk
from tqdm import tqdm
import json
import chime
from math import log2, prod
from random import randint

In [2]:
%load_ext chime

# Hyperparameters

The number of the most common words to be considered:

In [3]:
n_most_common = 10000

Minimum "collocation" below which no merging is done

In [4]:
colloc_threshold = 0.5

Min and max length of ngrams to be considered in the calculation (longer ngrams may result from merging with previously concatenated strings):

In [5]:
min_len = 2
max_len = 5

Batch size. If int, the number of tokens in a batch. If float, fraction of the total number of tokens in the text.

In [6]:
batch_size = 0.1

Weight to control contribution of frequency vs NPMI:

In [7]:
w = 0.5

Bias towards shorter ngrams

In [8]:
alpha = 0.5

In [9]:
with open('hyperparameters.json', 'w') as f:
    json.dump(
        {
            'n_most_common': n_most_common,
            'colloc_threshold': colloc_threshold,
            'min_len': min_len,
            'max_len': max_len,
            'batch_size': batch_size,
            'w': w,
            'alpha': alpha
        },
        f
    )

# Inputs:

Original corpus or the corpus with some tokens merged during previous iterations:

In [10]:
filename_corpus = 'legal_clauses.json'

Load the sequence with some tokens already merged in previous iterations if it exists or the original file if it does not:

In [11]:
try:
    with open('tokens.json', 'r') as f:
        tokens = json.load(f)
        print('Using tokens from previous iteraions')
except:
    with open(filename_corpus, 'r') as f:
        tokens = json.load(f)
        print('Starting with the original text')

Using tokens from previous iteraions


Load the log of merging operations previous iterations or create a new empty log:

In [12]:
try:
    with open('merging_log.json', 'r') as f:
        merging_log = json.load(f)
        print('Continuing merging log from previous iteraions')
except:
    merging_log = []
    print('Starting a new merging log')

Continuing merging log from previous iteraions


# Functions

In [13]:
def normalized_pmi(ngram, fdist_unigrams, fdist_ngrams, N_unigrams, N_ngrams):
    p1 = [fdist_unigrams[token] / N_unigrams for token in ngram]
    pn = fdist_ngrams[ngram] / N_ngrams
    return -log2(pn / prod(p1)) / (log2(pn) * (len(ngram) - 1) ** alpha)

In [14]:
def get_max(tokens, batch_size):
    len_tokens = len(tokens)
    '''The function returns a bigram with the max NPMI value and the NPMI'''
    if isinstance(batch_size, float):
        batch_len = int(len_tokens * batch_size)
    else:
        batch_len = batch_size
    # Select random slice
    start = randint(0, len_tokens - batch_len)
    slice = tokens[start: start + batch_len]
    fdist_unigrams = nltk.FreqDist(slice)
    fdist_ngrams = nltk.FreqDist(nltk.everygrams(slice, min_len=min_len, max_len=max_len))
    N_unigrams = fdist_unigrams.N()
    N_ngrams = fdist_ngrams.N()
    max_count = fdist_ngrams[fdist_ngrams.max()]
    common_bigrams = fdist_ngrams.most_common(n_most_common)
    max_colloc = 0
    max_ngram = None    
    for ngram, count in common_bigrams:
        npmi = normalized_pmi(ngram, fdist_unigrams, fdist_ngrams, N_unigrams, N_ngrams)
        rel_freq = fdist_ngrams[ngram] / max_count
        colloc = w * rel_freq + npmi
        if colloc > max_colloc:
            max_colloc = colloc
            max_ngram = ngram
            max_npmi = npmi
            max_rel_freq = rel_freq
    return {'ngram': max_ngram, 'rel_freq': max_rel_freq, 'npmi': max_npmi, 'colloc': max_colloc}

In [15]:
def merge(tokens, ngram):
    '''Merging all instances of the ngram in the sequence into a single token'''
    n = len(ngram)
    i = 0
    new_sequence = []
    ngram_merged = ' '.join(ngram)
    while i <= len(tokens) - n:
        if (tokens[i : i + n]) == list(ngram):
            new_sequence.append(ngram_merged)
            i += n
        else:
            new_sequence.append(tokens[i])
            i += 1
    # The last tokens
    for j in range(i, len(tokens) - 1):
        new_sequence.append(tokens[j])
    return new_sequence

# Merging
Original number of tokens:

In [16]:
len(tokens)

12355070

In [70]:
n_iter = 100

In [71]:
%%chime
for i in tqdm(range(n_iter)):
    ngram_to_merge = get_max(tokens, batch_size=batch_size)
    if ngram_to_merge['colloc'] < colloc_threshold:
        print('Collocation threshold is reached')
        break
    merging_log.append(ngram_to_merge)
    tokens = merge(tokens, ngram_to_merge['ngram'])

 45%|███████████████████████████████████████████████████████████████████▌                                                                                  | 45/100 [11:21<13:53, 15.15s/it]

Collocation threshold is reached


In [72]:
len(tokens)

11460242

Total iterations so far:

In [73]:
len(merging_log)

2042

In [74]:
with open('merging_log.json', 'w') as f:
    json.dump(merging_log, f)

In [75]:
with open('tokens.json', 'w') as f:
    json.dump(tokens, f)

# Remove single words

In [76]:
set_phrases = []
for token in tqdm(tokens):
    if len(token.split()) > 1:
        set_phrases.append(token)
set_phrases = list(set(set_phrases))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11460242/11460242 [00:05<00:00, 2126603.44it/s]


In [77]:
with open('set_phrases.json', 'w') as f:
    json.dump(set_phrases, f)

In [78]:
len(set_phrases)

2041

# Preliminary evaluation

In [79]:
import pandas as pd

In [80]:
df = pd.DataFrame({'set_prases': set_phrases})
df['word_count'] = df.set_prases.apply(lambda x: len(x.split()))
df.word_count.value_counts()

word_count
5     552
4     551
3     469
2     255
7      55
6      51
8      44
9      31
10     10
13      7
12      5
11      5
14      2
17      2
16      1
19      1
Name: count, dtype: int64

In [81]:
pd.DataFrame(merging_log)

,ngram,rel_freq,npmi,colloc
0,"[corporation, duly, organized, validly, existing]",0.002039,1.220702,1.221721
1,"[generally, accepted, accounting, principles]",0.002554,1.165551,1.166828
2,"[untrue, statement, or, alleged, untrue]",0.002199,1.073776,1.074876
3,"[have, a, material, adverse, effect]",0.015550,1.073189,1.080964
4,"[being, contested, in, good, faith]",0.001914,1.069853,1.070810
...,...,...,...,...
2037,"(itamar, medical)",0.002761,0.620129,0.621509
2038,"(shall, become, effective)",0.006598,0.504880,0.508179
2039,"(schedule, i, hereto)",0.003266,0.505592,0.507225
2040,"(fiduciary, duty)",0.002736,0.534237,0.535606


In [82]:
set_phrases[:100]

['including attorneys fees',
 'without the prior written consent',
 'act u s c et seq',
 'subject to the provisions of',
 'internal revenue code',
 'there is no',
 'on behalf of the holders',
 'the execution delivery or performance',
 'properties and to carry on its business as',
 'most recently',
 'in form and substance',
 'shall not affect the construction',
 'which the company or any of its subsidiaries is',
 'set forth opposite',
 'period of two years',
 'in its capacity as',
 'prior written consent of',
 'meanings ascribed to them',
 'have the respective meanings',
 'applicable laws and regulations',
 'in two or more',
 'agreement may not be modified',
 'this agreement may be modified',
 'will not be liable',
 'holders of a majority in aggregate principal amount of',
 'j p morgan',
 'actions suits',
 'as the case may be',
 'indemnify defend and hold',
 'with respect thereto',
 'or consequential damages',
 'a corporation duly organized validly existing and in good standing under th